In [39]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
import squarify  # Treemap을 위해 필요 (matplotlib 기반)

In [ ]:
# GitHub API 토큰 설정 (GitHub에서 Personal Access Token 생성 필요)
GITHUB_TOKEN = 'GITHUB_TOKEN'  # 여기에 토큰 입력
HEADERS = {'Authorization': f'token {GITHUB_TOKEN}'}

# 검색 쿼리 설정
QUERY = 'robotics'

# GitHub API 엔드포인트
SEARCH_URL = 'https://api.github.com/search/repositories'

In [54]:
# 키워드 해당되는 repo 가져오는 함수
def fetch_repos(query, sort='stars', order='desc', per_page=100):
    all_repos = []
    page = 1
    while True:
        params = {
            'q': query,
            'sort': sort,
            'order': order,
            'per_page': per_page,
            'page': page
        }
        response = requests.get(SEARCH_URL, headers=HEADERS, params=params)
        if response.status_code == 200:
            repos = response.json()['items']
            if not repos:
                break
            all_repos.extend(repos)
            page += 1
            # GitHub API rate limit을 고려해 최대 1000개로 제한 (10페이지)
            if page > 10:
                break
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break
    return all_repos

In [55]:
# repo 메타데이터 추출하는 함수
def get_repo_details(repo):
    return {
        'name': repo['full_name'],
        'stars': repo['stargazers_count'],
        'forks': repo['forks_count'],
        'language': repo['language'],
        'created_at': repo['created_at'],
        'topics': repo['topics']
    }

In [56]:
# 데이터 수집: 최대 1000개 리포지토리 가져오기
all_repos = fetch_repos(QUERY)

# 데이터프레임으로 변환
df = pd.DataFrame([get_repo_details(repo) for repo in all_repos])

# 날짜 형식 변환
df['created_at'] = pd.to_datetime(df['created_at'])
df['year'] = df['created_at'].dt.year